In [ ]:
from dataextract import readairfoil
from dataplotter import plotairfoil, compare_airfoils
import os
import numpy as np
from classifier_objects import Airfoil
import pickle

#Let this reading part opened while exe is started. So that, the datas will be ready to be used. 
data_path = '../data/train_data/NACAxxxx_Re_1000000'
save_path = '../data/train_data/NACAxxxx_Re_1000000_pkl/' 

airfoildata = readairfoil(data_path, save_path)


# airfoil_names = os.listdir(data_path)


"""
#The user should specify the range of lift coeff for range of alpha. The program will sort it for whole airfoil datas. ,
clcoeff_min = 1.1
alpha_range = [4, 7]
dCl_min = 0.10
cdcoeff_max = 0.11
clcd = 0.5


#It is written w.r.t xfoil output. Our VM should arrange this in better manner. 
max_index_finder = airfoildata[airfoil_names[12]]["AOA"] <= alpha_range[1]
min_index_finder = airfoildata[airfoil_names[12]]["AOA"] >= alpha_range[0]
index_finder = max_index_finder & min_index_finder
index_search = np.where(index_finder)[0]

# print(index_search)
condition_satisfied_airfoils_names = []

#The program will sort the airfoil datas for the range of lift coeff.
for airfoil_name in airfoil_names:
    if airfoildata[str(airfoil_name)]["data_available"] == True:
        clcoeff_max = airfoildata[str(airfoil_name)]["Cl"][index_search].max()
        dclcoeff_min = airfoildata[str(airfoil_name)]["dCl"][index_search].min()
        if clcoeff_max >= clcoeff_min and dclcoeff_min >= dCl_min:
            condition_satisfied_airfoils_names.append(airfoil_name)
    

print(condition_satisfied_airfoils_names)


#plotairfoil(airfoildata['NACA0012'])
#compare_airfoils(airfoildata['NACA0012'], airfoildata['NACA2423'])
"""

In [ ]:
with open('../data/train_data/NACAxxxx_Re_1000000_pkl/NACA0101.pkl', 'rb') as f:
    airfoil = pickle.load(f)
    print(airfoil.name)
    print(airfoil.polars[1000000][:, 0])
    print(airfoil.data_available)

In [1]:
from dataextract import readairfoil
from dataplotter import plot_polars, compare_airfoils, geometry_plotter
import os
import numpy as np
from classifier_objects import Airfoil
import pickle
from seach_foil import search_foil


#Let this reading part opened while exe is started. So that, the datas will be ready to be used.
data_path = '../data/train_data/'

clcoeff_min = 1.1
alpha_range = [5, 9]
dCl_min = 0.10
cdcoeff_max = 0.11
clcd = [0.5]
# , 
# airfoils = search_foil(1000000, alpha_range, data_path, clcoeff_min, cdcoeff_max, clcd, dCl_min)
airfoils = search_foil(1000000, alpha_range, data_path, Cl=1.4, Cd=0.2, dClda_range=[0, 0.2])
for i in airfoils:
    print(i.name)

i.plot_polars(1000000, "lift", "drag", "liftdrag")

i.NACA_geometry_generator()
i.plot_geometry(alpha_range)


ImportError: cannot import name 'plotairfoil' from 'dataplotter' (/mnt/c/Users/SARI/NADAS_codes/VekilModel/AirfoilClassifier/dataplotter.py)